# Extração de Características para Texto


Documentos são apresentados em formato textual. Tal formato, não pode ser compreendido por uma máquina e portanto, deve ser adaptado. À esta adaptação, de um formato textual para um formato numérico, atribuímos o nome extração de características (_feature extraction_).

## Bag of Words

## Tf-Idf

## Word Embeddings

## Referências

- [Machine Learning — Text Processing](https://towardsdatascience.com/machine-learning-text-processing-1d5a2d638958)